In [1]:
import torch
import torch.nn as nn
import torchvision
import math

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)


Using device: cuda


Roadmap:

Training and inference:
 * Call RPN layers
 * Generate Anchors
 * Convert anchors to proposals using Box transformation prediction
 * Filter Proposals

Training only:
 * Assign Ground Truth boxes to anchors
 * Compute labels and regression targets for anchors
 * Sample positive and negative anchors
 * Compute classification loss using sampled anchors
 * Compute localization loss using sampled positive anchors

In [3]:
def sample_positive_negative(labels, num_positive=256, num_negative=256):
    
    positive = torch.where(labels > 1)[0]
    negative=torch.where(labels == 0)[0]
    
    num_pos=num_positive
    num_pos=min(positive.numel(), num_pos)
    
    num_neg=num_negative 
    num_neg=min(negative.numel(), num_neg)
    
    perm_positive_ids=torch.randperm(positive.numel(), device=device)[:num_pos]
    perm_negative_ids=torch.randperm(negative.numel(), device=device)[:num_neg]
    
    pos_ids=positive[perm_positive_ids]
    neg_ids=negative[perm_negative_ids]
    
    sampled_pos_ids_mask = torch.zeros(labels.shape, dtype=torch.bool, device=device)
    sampled_neg_ids_mask = torch.zeros(labels.shape, dtype=torch.bool, device=device)
    
    sampled_pos_ids_mask[pos_ids] = True
    sampled_neg_ids_mask[neg_ids] = True    
    
    return sampled_pos_ids_mask, sampled_neg_ids_mask
def get_IOU(box1, box2):


    area1 = (box1[:, 2] - box1[:, 0]) * (box1[:, 3] - box1[:, 1])
    area2 = (box2[:, 2] - box2[:, 0]) * (box2[:, 3] - box2[:, 1]) 

    # top left x1,y1 and bottom right x2,y2
    x1 = torch.max(box1[:,None, 0], box2[:, 0])
    y1 = torch.max(box1[:,None, 1], box2[:, 1])

    x2 = torch.min(box1[:,None, 2], box2[:, 2])
    y2 = torch.min(box1[:,None, 3], box2[:, 3])
    
    intersection = (x2-x1).clamp(min=0) * (y2-y1).clamp(min=0)
    union = area1[:, None] + area2 - intersection
    iou = intersection / union
    return iou


def box_to_boundary(boxes,img_shape):
        boxes_x1 = boxes[..., 0]
        boxes_y1 = boxes[..., 1]
        boxes_x2 = boxes[..., 2]
        boxes_y2 = boxes[..., 3]

        height, width = img_shape[-2:]
        boxes_x1 = torch.clamp(boxes_x1,max=width)
        boxes_y1 = torch.clamp(boxes_y1,max=height)
        boxes_x2 = torch.clamp(boxes_x2,max=width)
        boxes_y2 = torch.clamp(boxes_y2,max=height)
        boxes=torch.cat((
            boxes_x1[..., None],
            boxes_y1[..., None],
            boxes_x2[..., None],
            boxes_y2[..., None]
        ),dim=-1)
        return boxes
class RegionalProposalNN(nn.Module):
    def __init__(self, num_classes=8, in_channels=512):
        
        super(RegionalProposalNN, self).__init__()
      


        self.scales=[128, 256, 512]  
        self.aspect_ratios = [0.5, 1.0, 2.0]
        self.num_anchors = len(self.scales) * len(self.aspect_ratios)

        # 3x3 convolutional layer for RPN
        self.rpn_conv = nn.Conv2d(in_channels,in_channels, kernel_size=3, padding=1,stride=1)
        # 1x1 convolutional layer for classification
        self.class_layer=nn.Conv2d(in_channels, self.num_anchors, kernel_size=1,stride=1)

        # 1x1 convolutional layer for bounding box regression
        self.bbox_layer = nn.Conv2d(in_channels, self.num_anchors * 4, kernel_size=1, stride=1)

    def anchors_to_predictions(self, predictions, anchors):
        bbox_predictions = predictions.reshape(predictions.size(0), -1, 4)
        
        # Get xs, cy , w , h from the predictions (x1, y1, x2, y2)
        w=anchors[:, 2] - anchors[:, 0]
        h=anchors[:, 3] - anchors[:, 1]
        cx = (anchors[:, 0] +  0.5 * w)
        cy = (anchors[:, 1] + 0.5 * h)
        dx= bbox_predictions[..., 0]
        dy= bbox_predictions[..., 1]
        dw= bbox_predictions[..., 2]
        dh= bbox_predictions[..., 3]

        pred_cx= dx * w[:,None] + cx[:, None]
        pred_cy= dy * h[:,None] + cy[:, None]
        pred_w = torch.exp(dw) * w[:, None]
        pred_h = torch.exp(dh) * h[:, None]

        pred_box_x1 = pred_cx - 0.5 * pred_w
        pred_box_y1 = pred_cy - 0.5 * pred_h
        pred_box_x2 = pred_cx + 0.5 * pred_w
        pred_box_y2 = pred_cy + 0.5 * pred_h

        pred_boxes= torch.stack([pred_box_x1, pred_box_y1, pred_box_x2, pred_box_y2], dim=2)

        return pred_boxes


    def transform_boxes_to_og_size(boxes,new_size,original_size):
        ratios = [ torch.tensor(s_og, dtype=torch.float32,devices=boxes.device) / torch.tensor(s,dtype=torch.float32, devices=boxes.device) for s_og, s in zip(original_size, new_size)]

        ratio_h, ratio_w = ratios
        xmin,ymin, xmax,ymax = boxes.unbind(1)
        xmin = xmin * ratio_w
        ymin = ymin * ratio_h
        xmax = xmax * ratio_w
        ymax = ymax * ratio_h
        return torch.stack([xmin, ymin, xmax, ymax], dim=1)

    def generate_anchors(self, image,feature):
        grid_h,grid_w=feature.shape[-2:]
        image_h, image_w = image.shape[-2:]

        stride_h = torch.tensor(image_h//grid_h, dtype=torch.float32, device=feature.device)
        stride_w = torch.tensor(image_w//grid_w, dtype=torch.float32, device=feature.device)
        aspect_ratios = torch.tensor(self.aspect_ratios, dtype=torch.float32, device=feature.device)
        scales = torch.tensor(self.scales, dtype=torch.float32, device=feature.device)

    




        # Make sure h/w = aspect_ratio and hxw=1

        h_ratios=torch.sqrt(aspect_ratios)
        w_ratios=1/h_ratios

        ws=(w_ratios[:,None] * scales[None,:]).view(-1)
        hs=(h_ratios[:,None] * scales[None,:]).view(-1)
        

        base_anchors = (torch.stack([-ws,-hs,ws,hs], dim=1) /2 ).round()

        # Get the shifts in the x and y axis

        shifts_x = torch.arange(0, grid_w,device=feature.device) * stride_w
        shifts_y = torch.arange(0, grid_h,device=feature.device) * stride_h

        shifts_x, shifts_y = torch.meshgrid(shifts_x, shifts_y, indexing='ij')

        shifts_x = shifts_x.reshape(-1)
        shifts_y = shifts_y.reshape(-1)
        shifts=torch.stack([shifts_x, shifts_y, shifts_x, shifts_y], dim=1)
        anchors=(shifts.view(1, -1, 4) + base_anchors.view(-1, 1, 4))
        anchors=anchors.reshape(-1,4)
        return anchors
    

    def filter_proposals(self, proposals, class_scores, img_shape):
        class_scores = class_scores.reshape(-1)
        class_scores = torch.sigmoid(class_scores)
        _, top_idx = class_scores.topk(10000)
        class_scores = class_scores[top_idx]
        proposals = proposals[top_idx]
        proposals = box_to_boundary(proposals, img_shape)

        # NMS
        keep_mask=torch.zeros_like(class_scores , dtype=torch.bool)
        keep_ids= torch.ops.torchvision.nms(proposals, class_scores, iou_threshold=0.7)             # IOU threshold 0.7
        post_nms_keep_indexes = keep_ids[class_scores[keep_ids].sort(descending=True)[1]]

        # Post NMS filtering
        proposals=proposals[post_nms_keep_indexes[:2000]]                                 # top 2000 proposals
        class_scores = class_scores[post_nms_keep_indexes[:2000]]
        return proposals, class_scores
    
    def assign_targets_to_anchors(self, anchors, gt_boxes):
        iou_matrix = get_IOU(anchors, gt_boxes)

        # Get the best ground truth box for each anchor
        best_match,best_gt_id = iou_matrix.max(dim=0)
        best_gt_id_pre_treshold = best_gt_id.clone()         # jeep a copy of the best_gt_id before thresholding


        below_threshold_mask = best_match < 0.3
        between_threshold_mask = (best_match >= 0.3) & (best_match < 0.7)
        best_gt_id[below_threshold_mask] = -1  # -1 for anchors that are below the threshold
        best_gt_id[between_threshold_mask] = -2


        # Low quality anchors
        best_anchor_iou_for_gt, _ = iou_matrix.max(dim=1)
        gt_pred_pair_max_iou=torch.where(iou_matrix == best_anchor_iou_for_gt[:, None])
        
        # Get all the anchor indexes
        preds_ids_to_update=gt_pred_pair_max_iou[1]
        best_gt_id[preds_ids_to_update]= best_gt_id_pre_treshold[preds_ids_to_update]

        # Best match index is either valid or -1 or -2
        matched_gt_boxes=gt_boxes[best_gt_id.clamp(min=0)]

        # Set all  foreground anchors to 1 and background anchors to 0
        labels = best_gt_id>=0
        labels=labels.to(  torch.float32)

        background_anchors= best_gt_id   == -1
        labels[background_anchors] = 0.0

        # anchors to be ignored to -1
        ignore_anchors = best_gt_id == -2
        labels[ignore_anchors] = -1.0

        return labels, matched_gt_boxes

    def boxes_to_transform_targets(self, groud_truth_boxes, anchors):
        #Get center x,y h,w from x1, y1, x2, y2 for  anchors 
        widths= anchors[:, 2] - anchors[:, 0]
        heights = anchors[:, 3] - anchors[:, 1]
        cx = (anchors[:, 0] + 0.5 * widths)
        cy = (anchors[:, 1] + 0.5 * heights)

        # for gt boxes
        gt_widths = groud_truth_boxes[:, 2] - groud_truth_boxes[:,0]
        gt_heights = groud_truth_boxes[:, 3] - groud_truth_boxes[:,1]
        gt_cx = (groud_truth_boxes[:, 0] + 0.5 * gt_widths)
        gt_cy = (groud_truth_boxes[:, 1] + 0.5 * gt_heights)


        target_dx = (gt_cx - cx) / widths
        target_dy = (gt_cy - cy) / heights
        target_dw = torch.log(gt_widths / widths)
        target_dh = torch.log(gt_heights / heights)

        regression_targets = torch.stack([target_dx, target_dy, target_dw, target_dh], dim=1)
        return regression_targets

    def forward(self, image, features,target):
        rpn_feat=nn.ReLU()(self.rpn_conv(features))
        classification_scores = self.class_layer(rpn_feat)
        bbox_predictions = self.bbox_layer(rpn_feat)

        anchors = self.generate_anchors(image, features)

        # class_scores = (Batch,anchors per location, h_feat, w_feat)
        anchors_per_location = classification_scores.shape[1]
        classification_scores = classification_scores.permute(0, 2, 3,1)
        classification_scores = classification_scores.reshape(-1,1)

        # classs_scores= (Batch*H_feat*w_feat, anchors per location,1)




        # bbox_predictions = (Batch,Anchors per location*4, h_feat, w_feat)
        bbox_predictions = bbox_predictions.view(bbox_predictions.size(0),
                                                 anchors_per_location,
                                                 4,
                                                 rpn_feat.shape[-2],
                                                 rpn_feat.shape[-1])
        bbox_predictions = bbox_predictions.permute(0, 3, 4, 1, 2)
        bbox_predictions = bbox_predictions.reshape(-1, 4)
        # bbox_predictions = (Batch*H_feat*w_feat, anchors per location,4)

        proposals=self.anchors_to_predictions(bbox_predictions.detach().reshape(-1,1,4), anchors)
        proposals = proposals.reshape(proposals.size(0), 4)


        proposals, class_scores = self.filter_proposals(proposals, classification_scores.detach(), image.shape)
        rpn_output = {
            'proposals': proposals,
            'class_scores': class_scores
        }
        if not self.training or target is None:
            return rpn_output
        else:  # assign ground truth boxes and  labels to anchors

            labels_for_anchors, matched_gt_boxes = self.assign_targets_to_anchors(anchors, target['bboxes'][0])

            regression_targets = self.boxes_to_transform_targets(matched_gt_boxes, anchors)

            # Sample positive and negative anchors for training
            sampled_pos_ids_mask, sampled_neg_ids_mask = sample_positive_negative(labels_for_anchors,128,128)

            sampled_ids=torch.where(sampled_pos_ids_mask | sampled_neg_ids_mask)[0]
            localization_loss  = nn.SmoothL1Loss(bbox_predictions[sampled_pos_ids_mask], regression_targets[sampled_ids],beta=1/9 , reductionn='sum') / sampled_ids.numel()

            classification_loss=nn.binary_cross_entropy_with_logits(
                classification_scores[sampled_ids].flatten(),
                labels_for_anchors[sampled_ids].flatten(),
                
            ) 

            rpn_output['rpn_classificatoin_loss'] = classification_loss
            rpn_output['rpn_localization_loss'] = localization_loss

            return rpn_output


ROI head road map:

Training:
* Assign ground truth boxes to proposals

* Sample posotive and negative proposals
* Get classification and regression targets for proposals
* ROI pooling to get proposal features
* Call classification and regression layers
* Compute classification and localization loss

Inference:
* ROI pooling to get proposal features
* Classification and regression
* Convert proposals to predictions with box transformation prediction
* Filter boxes
    

In [4]:
class ROIHead(nn.Module):
    def __init__(self, num_classes=8, in_channels=512):
        super(ROIHead, self).__init__()
        self.num_classes = num_classes
        self.pool_size = 7
        self.fc_inner_dim = 1024

        self.fc1=nn.Linear(in_channels*self.pool_size*self.pool_size, self.fc_inner_dim)

        self.fc2=nn.Linear(self.fc_inner_dim, self.fc_inner_dim)
        self.class_layer = nn.Linear(self.fc_inner_dim, num_classes)
        self.bbox_reg_layer = nn.Linear(self.fc_inner_dim, num_classes * 4)


    def assign_targets_to_proposals(self, proposals, gt_boxes, gt_labels):
        iou_matrix = get_IOU(proposals, gt_boxes)

        # Get the best ground truth box for each proposal
        best_match, best_gt_id = iou_matrix.max(dim=0)
        
        below_low_threshold_mask = best_match < 0.5
        best_gt_id[below_low_threshold_mask] = -1  # -1 for proposals that are below the threshold
        matched_gt_boxes = gt_boxes[best_gt_id.clamp(min=0)]
        labels=gt_labels(best_gt_id.clamp(min=0))
        labels=labels.to(torch.int64)

        background_proposals = best_gt_id == -1
        labels[background_proposals] = 0  # Background proposals are labeled as 0
        return labels, matched_gt_boxes

    def filter_predictions(self, pred_boxes,pred_labels,pred_scores):
        # REmove low scoring boxes
        keep_mask = torch.where(pred_scores > 0.05)[0]
        pred_boxes,pred_scores,pred_labels = pred_boxes[keep_mask], pred_scores[keep_mask], pred_labels[keep_mask]

        # NMS
        keep_mask = torch.zeros_like(pred_scores, dtype=torch.bool)
        for class_id in torch.unique(pred_labels):
            ids= torch.where(pred_labels == class_id)[0]
            keep_ids= torch.ops.torchvision.nms(
                pred_boxes[ids], pred_scores[ids], iou_threshold=0.5)

            keep_mask[ids[keep_ids]] = True
        keep_indices = torch.where(keep_mask)[0]
        post_nms_indices = keep_indices[pred_scores[keep_indices].sort(descending=True)[1]]
        keep=post_nms_indices[:100]  # Keep top 100 predictions
        return pred_boxes[keep], pred_labels[keep], pred_scores[keep] 





    def forward(self, features, proposals, image_shape,target):
        if self.training and target is not None:
            # Assign ground truth boxes to proposals
            gt_boxes = target['boxes'][0]
            gt_labels = target['labels'][0]

            # Assign labels and gt boxes to proposals

            labels,matched_gt_boxes = self.assign_targets_to_proposals(proposals, gt_boxes, gt_labels)
            
            
            sampled_neg_ids_mask, sampled_pos_ids_mask = sample_positive_negative(labels, 32, 128-32) 
            # Sample positive and negative proposals for training
            
            sampled_ids = torch.where(sampled_pos_ids_mask | sampled_neg_ids_mask)[0]

            proposals = proposals[sampled_ids]
            labels = labels[sampled_ids]
            matched_gt_boxes = matched_gt_boxes[sampled_ids]

            regression_targets = self.boxes_to_transform_targets(matched_gt_boxes, proposals)  

        # ROI Pooling
        spacial_scale= 0.0625

        proposal_roi_pool_feats = torchvision.ops.roi_pool(
            features, proposals, output_size=self.pool_size, spatial_scale=spacial_scale)
        
        
        proposal_roi_pool_feats = proposal_roi_pool_feats.flatten(start_dim=1)

        box_fc1=torch.nn.functional.relu(self.fc1(proposal_roi_pool_feats))
        box_fc2 = torch.nn.functional.relu(self.fc2(box_fc1))

        class_scores = self.class_layer(box_fc2)
        bbox_predictions = self.bbox_reg_layer(box_fc2)

        num_boxes, num_classes = class_scores.shape
        bbox_predictions = bbox_predictions.reshape(num_boxes, num_classes, 4)

        frcnn_output = {}

        if self.training and target is not None:
            classification_loss = torch.nn.functional.cross_entropy(class_scores, labels)

            #compute localization loss only for non background proposals
            fg_proposal_ids=torch.where(labels > 0)[0]

            fg_class_labels = labels[fg_proposal_ids]
            localization_loss = torch.nn.functional.smooth_l1_loss(
                bbox_predictions[fg_proposal_ids, fg_class_labels],
                regression_targets[fg_proposal_ids],
                beta=1 / 9,
                reduction='sum'
            ) / fg_proposal_ids.numel()

            frcnn_output['frcnn_classification_loss'] = classification_loss
            frcnn_output['frcnn_localization_loss'] = localization_loss
            return frcnn_output
        else:
            #apply transformation to the proposals
            pred_boxes = apply_regression_to_proposals(bbox_predictions, proposals)

            pred_scores = torch.nn.functional.softmax(class_scores, dim=1)

            #clamp boxes to image boundaries
            pred_boxes = box_to_boundary(pred_boxes, image_shape)

            # create labels for predictions
            pred_labels = torch.arrange(num_classes, device=pred_boxes.device)
            pred_labels= pred_labels.view(1, -1).expand_as(pred_scores)


            # remove background predictions

            pred_boxes = pred_boxes[:, 1:]
            pred_scores = pred_scores[:, 1:]
            pred_labels = pred_labels[:, 1:]

            # batch everything making every class a separate prediction
            pred_boxes = pred_boxes.reshape(-1, 4)
            pred_scores = pred_scores.reshape(-1)
            pred_labels = pred_labels.reshape(-1)
            
            pred_boxes, pred_labels, pred_scores = self.filter_predictions(pred_boxes, pred_labels, pred_scores)
            
            frcnn_output['pred_boxes'] = pred_boxes
            frcnn_output['pred_labels'] = pred_labels
            frcnn_output['pred_scores'] = pred_scores

            return frcnn_output

The Faster RCNN is comprised of a pretrained vgg16 backbone and the regional proposal nn and roi head we implemented above

In [5]:
class FasterRCNN(nn.Module):
    def __init__(self, num_classes=8):
        super(FasterRCNN, self).__init__()
        vgg16=torchvision.models.vgg16(pretrained=True)
        self.backbone = vgg16.features[:-1] # Exclude the last max pooling layer
        self.rpn = RegionalProposalNN(num_classes=num_classes, in_channels=512)
        self.roi_head = ROIHead(num_classes=num_classes, in_channels=512)

        for layer in self.backbone[:10]:
            for param in layer.parameters():
                param.requires_grad = False
        self.image_mean= [0.485, 0.456, 0.406]
        self.image_std = [0.229, 0.224, 0.225]
        self.min_size = 600
        self.max_size = 1000
    def normalize_resize(self, image,bboxes=None):
        # Normalize the image
        mean= torch.as_tensor(self.image_mean, dtype=image.dtype, device=image.device)
        std = torch.as_tensor(self.image_std, dtype=image.dtype, device=image.device)
        image = (image - mean[:, None, None]) / std[:, None, None]

        # resize the image so that lower dim gets to 600 but larger dim does not exceed 1000

        h, w = image.shape[-2:]
        im_shape= torch.tensor(image.shape[-2:])
        min_size = torch.min(im_shape).to(torch.float32)
        max_size = torch.max(im_shape).to(torch.float32)
        scale=torch.min(
            float(self.min_size) / min_size,     
            float(self.max_size) / max_size
        )
        image = torch.nn.functional.interpolate(
            image,
            scale_factor=scale,
            mode='bilinear',
            recompute_scale_factor=True, 
            align_corners=False
        )

        # Resize the bounding boxes if provided
        if bboxes is not None:
            ratios = [ torch.tensor(s,dtype=torch.float32,device=bboxes.device)/
                      torch.tensor(s_orig,dtype=torch.float32,device=bboxes.device)
                     for s, s_orig in zip(image.shape[-2:], (h,w))]
            
            ratio_h,ratio_w=ratios
            xmin, ymin, xmax, ymax = bboxes.unbind(2)
            xmin = xmin * ratio_w
            ymin = ymin * ratio_h
            xmax = xmax * ratio_w
            ymax = ymax * ratio_h

            bboxes = torch.stack([xmin, ymin, xmax, ymax], dim=2)
            return image, bboxes



    def forward(self, image, target=None):
        old_shape = image.shape[-2:]
        if self.training:
            image, bboxes = self.normalize_resize(image, target['bboxes'])
            target['bboxes'] = bboxes
        else:
            image, _ = self.normalize_resize(image,None)

         # call backbone and RPN   
        features = self.backbone(image)
        rpn_output = self.rpn(image, features, target)
        proposals = rpn_output['proposals']

        FasterRCNN_output = self.roi_head(features, proposals, image.shape[-2:])

        if not self.training:
            # transform the predicted boxes to the original image shape
            FasterRCNN_output['boxes'] = transform_boxes_to_og_size(
                FasterRCNN_output['boxes'], image.shape[-2:], old_shape
            )

        return rpn_output, FasterRCNN_output

In [3]:

import torch
import torch.nn as nn
import torchvision
import math

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


def get_iou(boxes1, boxes2):
    r"""
    IOU between two sets of boxes
    :param boxes1: (Tensor of shape N x 4)
    :param boxes2: (Tensor of shape M x 4)
    :return: IOU matrix of shape N x M
    """
    # Area of boxes (x2-x1)*(y2-y1)
    area1 = (boxes1[:, 2] - boxes1[:, 0]) * (boxes1[:, 3] - boxes1[:, 1])  # (N,)
    area2 = (boxes2[:, 2] - boxes2[:, 0]) * (boxes2[:, 3] - boxes2[:, 1])  # (M,)
    
    # Get top left x1,y1 coordinate
    x_left = torch.max(boxes1[:, None, 0], boxes2[:, 0])  # (N, M)
    y_top = torch.max(boxes1[:, None, 1], boxes2[:, 1])  # (N, M)
    
    # Get bottom right x2,y2 coordinate
    x_right = torch.min(boxes1[:, None, 2], boxes2[:, 2])  # (N, M)
    y_bottom = torch.min(boxes1[:, None, 3], boxes2[:, 3])  # (N, M)
    
    intersection_area = (x_right - x_left).clamp(min=0) * (y_bottom - y_top).clamp(min=0)  # (N, M)
    union = area1[:, None] + area2 - intersection_area  # (N, M)
    iou = intersection_area / union  # (N, M)
    return iou


def boxes_to_transformation_targets(ground_truth_boxes, anchors_or_proposals):
    r"""
    Given all anchor boxes or proposals in image and their respective
    ground truth assignments, we use the x1,y1,x2,y2 coordinates of them
    to get tx,ty,tw,th transformation targets for all anchor boxes or proposals
    :param ground_truth_boxes: (anchors_or_proposals_in_image, 4)
        Ground truth box assignments for the anchors/proposals
    :param anchors_or_proposals: (anchors_or_proposals_in_image, 4) Anchors/Proposal boxes
    :return: regression_targets: (anchors_or_proposals_in_image, 4) transformation targets tx,ty,tw,th
        for all anchors/proposal boxes
    """
    
    # Get center_x,center_y,w,h from x1,y1,x2,y2 for anchors
    widths = anchors_or_proposals[:, 2] - anchors_or_proposals[:, 0]
    heights = anchors_or_proposals[:, 3] - anchors_or_proposals[:, 1]
    center_x = anchors_or_proposals[:, 0] + 0.5 * widths
    center_y = anchors_or_proposals[:, 1] + 0.5 * heights
    
    # Get center_x,center_y,w,h from x1,y1,x2,y2 for gt boxes
    gt_widths = ground_truth_boxes[:, 2] - ground_truth_boxes[:, 0]
    gt_heights = ground_truth_boxes[:, 3] - ground_truth_boxes[:, 1]
    gt_center_x = ground_truth_boxes[:, 0] + 0.5 * gt_widths
    gt_center_y = ground_truth_boxes[:, 1] + 0.5 * gt_heights
    
    targets_dx = (gt_center_x - center_x) / widths
    targets_dy = (gt_center_y - center_y) / heights
    targets_dw = torch.log(gt_widths / widths)
    targets_dh = torch.log(gt_heights / heights)
    regression_targets = torch.stack((targets_dx, targets_dy, targets_dw, targets_dh), dim=1)
    return regression_targets


def apply_regression_pred_to_anchors_or_proposals(box_transform_pred, anchors_or_proposals):
    r"""
    Given the transformation parameter predictions for all
    input anchors or proposals, transform them accordingly
    to generate predicted proposals or predicted boxes
    :param box_transform_pred: (num_anchors_or_proposals, num_classes, 4)
    :param anchors_or_proposals: (num_anchors_or_proposals, 4)
    :return pred_boxes: (num_anchors_or_proposals, num_classes, 4)
    """
    box_transform_pred = box_transform_pred.reshape(
        box_transform_pred.size(0), -1, 4)
    
    # Get cx, cy, w, h from x1,y1,x2,y2
    w = anchors_or_proposals[:, 2] - anchors_or_proposals[:, 0]
    h = anchors_or_proposals[:, 3] - anchors_or_proposals[:, 1]
    center_x = anchors_or_proposals[:, 0] + 0.5 * w
    center_y = anchors_or_proposals[:, 1] + 0.5 * h
    
    dx = box_transform_pred[..., 0]
    dy = box_transform_pred[..., 1]
    dw = box_transform_pred[..., 2]
    dh = box_transform_pred[..., 3]
    # dh -> (num_anchors_or_proposals, num_classes)
    
    # Prevent sending too large values into torch.exp()
    dw = torch.clamp(dw, max=math.log(1000.0 / 16))
    dh = torch.clamp(dh, max=math.log(1000.0 / 16))
    
    pred_center_x = dx * w[:, None] + center_x[:, None]
    pred_center_y = dy * h[:, None] + center_y[:, None]
    pred_w = torch.exp(dw) * w[:, None]
    pred_h = torch.exp(dh) * h[:, None]
    # pred_center_x -> (num_anchors_or_proposals, num_classes)
    
    pred_box_x1 = pred_center_x - 0.5 * pred_w
    pred_box_y1 = pred_center_y - 0.5 * pred_h
    pred_box_x2 = pred_center_x + 0.5 * pred_w
    pred_box_y2 = pred_center_y + 0.5 * pred_h
    
    pred_boxes = torch.stack((
        pred_box_x1,
        pred_box_y1,
        pred_box_x2,
        pred_box_y2),
        dim=2)
    # pred_boxes -> (num_anchors_or_proposals, num_classes, 4)
    return pred_boxes


def sample_positive_negative(labels, positive_count, total_count):
    # Sample positive and negative proposals
    positive = torch.where(labels >= 1)[0]
    negative = torch.where(labels == 0)[0]
    num_pos = positive_count
    num_pos = min(positive.numel(), num_pos)
    num_neg = total_count - num_pos
    num_neg = min(negative.numel(), num_neg)
    perm_positive_idxs = torch.randperm(positive.numel(),
                                        device=positive.device)[:num_pos]
    perm_negative_idxs = torch.randperm(negative.numel(),
                                        device=negative.device)[:num_neg]
    pos_idxs = positive[perm_positive_idxs]
    neg_idxs = negative[perm_negative_idxs]
    sampled_pos_idx_mask = torch.zeros_like(labels, dtype=torch.bool)
    sampled_neg_idx_mask = torch.zeros_like(labels, dtype=torch.bool)
    sampled_pos_idx_mask[pos_idxs] = True
    sampled_neg_idx_mask[neg_idxs] = True
    return sampled_neg_idx_mask, sampled_pos_idx_mask


def clamp_boxes_to_image_boundary(boxes, image_shape):
    boxes_x1 = boxes[..., 0]
    boxes_y1 = boxes[..., 1]
    boxes_x2 = boxes[..., 2]
    boxes_y2 = boxes[..., 3]
    height, width = image_shape[-2:]
    boxes_x1 = boxes_x1.clamp(min=0, max=width)
    boxes_x2 = boxes_x2.clamp(min=0, max=width)
    boxes_y1 = boxes_y1.clamp(min=0, max=height)
    boxes_y2 = boxes_y2.clamp(min=0, max=height)
    boxes = torch.cat((
        boxes_x1[..., None],
        boxes_y1[..., None],
        boxes_x2[..., None],
        boxes_y2[..., None]),
        dim=-1)
    return boxes


def transform_boxes_to_original_size(boxes, new_size, original_size):
    r"""
    Boxes are for resized image (min_size=600, max_size=1000).
    This method converts the boxes to whatever dimensions
    the image was before resizing
    :param boxes:
    :param new_size:
    :param original_size:
    :return:
    """
    ratios = [
        torch.tensor(s_orig, dtype=torch.float32, device=boxes.device)
        / torch.tensor(s, dtype=torch.float32, device=boxes.device)
        for s, s_orig in zip(new_size, original_size)
    ]
    ratio_height, ratio_width = ratios
    xmin, ymin, xmax, ymax = boxes.unbind(1)
    xmin = xmin * ratio_width
    xmax = xmax * ratio_width
    ymin = ymin * ratio_height
    ymax = ymax * ratio_height
    return torch.stack((xmin, ymin, xmax, ymax), dim=1)


class RegionProposalNetwork(nn.Module):
    r"""
    RPN with following layers on the feature map
        1. 3x3 conv layer followed by Relu
        2. 1x1 classification conv with num_anchors(num_scales x num_aspect_ratios) output channels
        3. 1x1 classification conv with 4 x num_anchors output channels

    Classification is done via one value indicating probability of foreground
    with sigmoid applied during inference
    """
    
    def __init__(self, in_channels, scales, aspect_ratios, model_config):
        super(RegionProposalNetwork, self).__init__()
        self.scales = scales
        self.low_iou_threshold = model_config['rpn_bg_threshold']
        self.high_iou_threshold = model_config['rpn_fg_threshold']
        self.rpn_nms_threshold = model_config['rpn_nms_threshold']
        self.rpn_batch_size = model_config['rpn_batch_size']
        self.rpn_pos_count = int(model_config['rpn_pos_fraction'] * self.rpn_batch_size)
        self.rpn_topk = model_config['rpn_train_topk'] if self.training else model_config['rpn_test_topk']
        self.rpn_prenms_topk = model_config['rpn_train_prenms_topk'] if self.training \
            else model_config['rpn_test_prenms_topk']
        self.aspect_ratios = aspect_ratios
        self.num_anchors = len(self.scales) * len(self.aspect_ratios)
        
        # 3x3 conv layer
        self.rpn_conv = nn.Conv2d(in_channels, in_channels, kernel_size=3, stride=1, padding=1)
        
        # 1x1 classification conv layer
        self.cls_layer = nn.Conv2d(in_channels, self.num_anchors, kernel_size=1, stride=1)
        
        # 1x1 regression
        self.bbox_reg_layer = nn.Conv2d(in_channels, self.num_anchors * 4, kernel_size=1, stride=1)
        
        for layer in [self.rpn_conv, self.cls_layer, self.bbox_reg_layer]:
            torch.nn.init.normal_(layer.weight, std=0.01)
            torch.nn.init.constant_(layer.bias, 0)
    
    def generate_anchors(self, image, feat):
        r"""
        Method to generate anchors. First we generate one set of zero-centred anchors
        using the scales and aspect ratios provided.
        We then generate shift values in x,y axis for all featuremap locations.
        The single zero centred anchors generated are replicated and shifted accordingly
        to generate anchors for all feature map locations.
        Note that these anchors are generated such that their centre is top left corner of the
        feature map cell rather than the centre of the feature map cell.
        :param image: (N, C, H, W) tensor
        :param feat: (N, C_feat, H_feat, W_feat) tensor
        :return: anchor boxes of shape (H_feat * W_feat * num_anchors_per_location, 4)
        """
        grid_h, grid_w = feat.shape[-2:]
        image_h, image_w = image.shape[-2:]
        
        # For the vgg16 case stride would be 16 for both h and w
        stride_h = torch.tensor(image_h // grid_h, dtype=torch.int64, device=feat.device)
        stride_w = torch.tensor(image_w // grid_w, dtype=torch.int64, device=feat.device)
        
        scales = torch.as_tensor(self.scales, dtype=feat.dtype, device=feat.device)
        aspect_ratios = torch.as_tensor(self.aspect_ratios, dtype=feat.dtype, device=feat.device)
        
        # Assuming anchors of scale 128 sq pixels
        # For 1:1 it would be (128, 128) -> area=16384
        # For 2:1 it would be (181.02, 90.51) -> area=16384
        # For 1:2 it would be (90.51, 181.02) -> area=16384
        
        # The below code ensures h/w = aspect_ratios and h*w=1
        h_ratios = torch.sqrt(aspect_ratios)
        w_ratios = 1 / h_ratios
        
        # Now we will just multiply h and w with scale(example 128)
        # to make h*w = 128 sq pixels and h/w = aspect_ratios
        # This gives us the widths and heights of all anchors
        # which we need to replicate at all locations
        ws = (w_ratios[:, None] * scales[None, :]).view(-1)
        hs = (h_ratios[:, None] * scales[None, :]).view(-1)
        
        # Now we make all anchors zero centred
        # So x1, y1, x2, y2 = -w/2, -h/2, w/2, h/2
        base_anchors = torch.stack([-ws, -hs, ws, hs], dim=1) / 2
        base_anchors = base_anchors.round()
        
        # Get the shifts in x axis (0, 1,..., W_feat-1) * stride_w
        shifts_x = torch.arange(0, grid_w, dtype=torch.int32, device=feat.device) * stride_w

        # Get the shifts in x axis (0, 1,..., H_feat-1) * stride_h
        shifts_y = torch.arange(0, grid_h, dtype=torch.int32, device=feat.device) * stride_h
        
        # Create a grid using these shifts
        shifts_y, shifts_x = torch.meshgrid(shifts_y, shifts_x, indexing="ij")
        # shifts_x -> (H_feat, W_feat)
        # shifts_y -> (H_feat, W_feat)
        
        shifts_x = shifts_x.reshape(-1)
        shifts_y = shifts_y.reshape(-1)
        # Setting shifts for x1 and x2(same as shifts_x) and y1 and y2(same as shifts_y)
        shifts = torch.stack((shifts_x, shifts_y, shifts_x, shifts_y), dim=1)
        # shifts -> (H_feat * W_feat, 4)
        
        # base_anchors -> (num_anchors_per_location, 4)
        # shifts -> (H_feat * W_feat, 4)
        # Add these shifts to each of the base anchors
        anchors = (shifts.view(-1, 1, 4) + base_anchors.view(1, -1, 4))
        # anchors -> (H_feat * W_feat, num_anchors_per_location, 4)
        anchors = anchors.reshape(-1, 4)
        # anchors -> (H_feat * W_feat * num_anchors_per_location, 4)
        return anchors
    
    def assign_targets_to_anchors(self, anchors, gt_boxes):
        r"""
        For each anchor assign a ground truth box based on the IOU.
        Also creates classification labels to be used for training
        label=1 for anchors where maximum IOU with a gtbox > high_iou_threshold
        label=0 for anchors where maximum IOU with a gtbox < low_iou_threshold
        label=-1 for anchors where maximum IOU with a gtbox between (low_iou_threshold, high_iou_threshold)
        :param anchors: (num_anchors_in_image, 4) all anchor boxes
        :param gt_boxes: (num_gt_boxes_in_image, 4) all ground truth boxes
        :return:
            label: (num_anchors_in_image) {-1/0/1}
            matched_gt_boxes: (num_anchors_in_image, 4) coordinates of assigned gt_box to each anchor
                Even background/to_be_ignored anchors will be assigned some ground truth box.
                It's fine, we will use label to differentiate those instances later
        """
        
        # Get (gt_boxes, num_anchors_in_image) IOU matrix
        iou_matrix = get_iou(gt_boxes, anchors)
        
        # For each anchor get the gt box index with maximum overlap
        best_match_iou, best_match_gt_idx = iou_matrix.max(dim=0)
        # best_match_gt_idx -> (num_anchors_in_image)
        
        # This copy of best_match_gt_idx will be needed later to
        # add low quality matches
        best_match_gt_idx_pre_thresholding = best_match_gt_idx.clone()
        
        # Based on threshold, update the values of best_match_gt_idx
        # For anchors with highest IOU < low_threshold update to be -1
        # For anchors with highest IOU between low_threshold & high threshold update to be -2
        below_low_threshold = best_match_iou < self.low_iou_threshold
        between_thresholds = (best_match_iou >= self.low_iou_threshold) & (best_match_iou < self.high_iou_threshold)
        best_match_gt_idx[below_low_threshold] = -1
        best_match_gt_idx[between_thresholds] = -2
        
        # Add low quality anchor boxes, if for a given ground truth box, these are the ones
        # that have highest IOU with that gt box
        
        # For each gt box, get the maximum IOU value amongst all anchors
        best_anchor_iou_for_gt, _ = iou_matrix.max(dim=1)
        # best_anchor_iou_for_gt -> (num_gt_boxes_in_image)
        
        # For each gt box get those anchors
        # which have this same IOU as present in best_anchor_iou_for_gt
        # This is to ensure if 10 anchors all have the same IOU value,
        # which is equal to the highest IOU that this gt box has with any anchor
        # then we get all these 10 anchors
        gt_pred_pair_with_highest_iou = torch.where(iou_matrix == best_anchor_iou_for_gt[:, None])
        # gt_pred_pair_with_highest_iou -> [0, 0, 0, 1, 1, 1], [8896,  8905,  8914, 10472, 10805, 11138]
        # This means that anchors at the first 3 indexes have an IOU with gt box at index 0
        # which is equal to the highest IOU that this gt box has with ANY anchor
        # Similarly anchor at last three indexes(10472, 10805, 11138) have an IOU with gt box at index 1
        # which is equal to the highest IOU that this gt box has with ANY anchor
        # These 6 anchor indexes will also be added as positive anchors
        
        # Get all the anchors indexes to update
        pred_inds_to_update = gt_pred_pair_with_highest_iou[1]
        
        # Update the matched gt index for all these anchors with whatever was the best gt box
        # prior to thresholding
        best_match_gt_idx[pred_inds_to_update] = best_match_gt_idx_pre_thresholding[pred_inds_to_update]
        
        # best_match_gt_idx is either a valid index for all anchors or -1(background) or -2(to be ignored)
        # Clamp this so that the best_match_gt_idx is a valid non-negative index
        # At this moment the -1 and -2 labelled anchors will be mapped to the 0th gt box
        matched_gt_boxes = gt_boxes[best_match_gt_idx.clamp(min=0)]
        
        # Set all foreground anchor labels as 1
        labels = best_match_gt_idx >= 0
        labels = labels.to(dtype=torch.float32)
        
        # Set all background anchor labels as 0
        background_anchors = best_match_gt_idx == -1
        labels[background_anchors] = 0.0
        
        # Set all to be ignored anchor labels as -1
        ignored_anchors = best_match_gt_idx == -2
        labels[ignored_anchors] = -1.0
        # Later for classification we will only pick labels which have > 0 label
        
        return labels, matched_gt_boxes

    def filter_proposals(self, proposals, cls_scores, image_shape):
        r"""
        This method does three kinds of filtering/modifications
        1. Pre NMS topK filtering
        2. Make proposals valid by clamping coordinates(0, width/height)
        2. Small Boxes filtering based on width and height
        3. NMS
        4. Post NMS topK filtering
        :param proposals: (num_anchors_in_image, 4)
        :param cls_scores: (num_anchors_in_image, 4) these are cls logits
        :param image_shape: resized image shape needed to clip proposals to image boundary
        :return: proposals and cls_scores: (num_filtered_proposals, 4) and (num_filtered_proposals)
        """
        # Pre NMS Filtering
        cls_scores = cls_scores.reshape(-1)
        cls_scores = torch.sigmoid(cls_scores)
        _, top_n_idx = cls_scores.topk(min(self.rpn_prenms_topk, len(cls_scores)))
        
        cls_scores = cls_scores[top_n_idx]
        proposals = proposals[top_n_idx]
        ##################
        
        # Clamp boxes to image boundary
        proposals = clamp_boxes_to_image_boundary(proposals, image_shape)
        ####################
        
        # Small boxes based on width and height filtering
        min_size = 16
        ws, hs = proposals[:, 2] - proposals[:, 0], proposals[:, 3] - proposals[:, 1]
        keep = (ws >= min_size) & (hs >= min_size)
        keep = torch.where(keep)[0]
        proposals = proposals[keep]
        cls_scores = cls_scores[keep]
        ####################
        
        # NMS based on objectness scores
        keep_mask = torch.zeros_like(cls_scores, dtype=torch.bool)
        keep_indices = torch.ops.torchvision.nms(proposals, cls_scores, self.rpn_nms_threshold)
        keep_mask[keep_indices] = True
        keep_indices = torch.where(keep_mask)[0]
        # Sort by objectness
        post_nms_keep_indices = keep_indices[cls_scores[keep_indices].sort(descending=True)[1]]
        
        # Post NMS topk filtering
        proposals, cls_scores = (proposals[post_nms_keep_indices[:self.rpn_topk]],
                                 cls_scores[post_nms_keep_indices[:self.rpn_topk]])
        
        return proposals, cls_scores
    
    def forward(self, image, feat, target=None):
        r"""
        Main method for RPN does the following:
        1. Call RPN specific conv layers to generate classification and
            bbox transformation predictions for anchors
        2. Generate anchors for entire image
        3. Transform generated anchors based on predicted bbox transformation to generate proposals
        4. Filter proposals
        5. For training additionally we do the following:
            a. Assign target ground truth labels and boxes to each anchors
            b. Sample positive and negative anchors
            c. Compute classification loss using sampled pos/neg anchors
            d. Compute Localization loss using sampled pos anchors
        :param image:
        :param feat:
        :param target:
        :return:
        """
        # Call RPN layers
        rpn_feat = nn.ReLU()(self.rpn_conv(feat))
        cls_scores = self.cls_layer(rpn_feat)
        box_transform_pred = self.bbox_reg_layer(rpn_feat)

        # Generate anchors
        anchors = self.generate_anchors(image, feat)
        
        # Reshape classification scores to be (Batch Size * H_feat * W_feat * Number of Anchors Per Location, 1)
        # cls_score -> (Batch_Size, Number of Anchors per location, H_feat, W_feat)
        number_of_anchors_per_location = cls_scores.size(1)
        cls_scores = cls_scores.permute(0, 2, 3, 1)
        cls_scores = cls_scores.reshape(-1, 1)
        # cls_score -> (Batch_Size*H_feat*W_feat*Number of Anchors per location, 1)
        
        # Reshape bbox predictions to be (Batch Size * H_feat * W_feat * Number of Anchors Per Location, 4)
        # box_transform_pred -> (Batch_Size, Number of Anchors per location*4, H_feat, W_feat)
        box_transform_pred = box_transform_pred.view(
            box_transform_pred.size(0),
            number_of_anchors_per_location,
            4,
            rpn_feat.shape[-2],
            rpn_feat.shape[-1])
        box_transform_pred = box_transform_pred.permute(0, 3, 4, 1, 2)
        box_transform_pred = box_transform_pred.reshape(-1, 4)
        # box_transform_pred -> (Batch_Size*H_feat*W_feat*Number of Anchors per location, 4)
        
        # Transform generated anchors according to box transformation prediction
        proposals = apply_regression_pred_to_anchors_or_proposals(
            box_transform_pred.detach().reshape(-1, 1, 4),
            anchors)
        proposals = proposals.reshape(proposals.size(0), 4)
        ######################
        
        proposals, scores = self.filter_proposals(proposals, cls_scores.detach(), image.shape)
        rpn_output = {
            'proposals': proposals,
            'scores': scores
        }
        if not self.training or target is None:
            # If we are not training no need to do anything
            return rpn_output
        else:
            # Assign gt box and label for each anchor
            labels_for_anchors, matched_gt_boxes_for_anchors = self.assign_targets_to_anchors(
                anchors,
                target['bboxes'][0])
            
            # Based on gt assignment above, get regression target for the anchors
            # matched_gt_boxes_for_anchors -> (Number of anchors in image, 4)
            # anchors -> (Number of anchors in image, 4)
            regression_targets = boxes_to_transformation_targets(matched_gt_boxes_for_anchors, anchors)
            
            ####### Sampling positive and negative anchors ####
            # Our labels were {fg:1, bg:0, to_be_ignored:-1}
            sampled_neg_idx_mask, sampled_pos_idx_mask = sample_positive_negative(
                labels_for_anchors,
                positive_count=self.rpn_pos_count,
                total_count=self.rpn_batch_size)
            
            sampled_idxs = torch.where(sampled_pos_idx_mask | sampled_neg_idx_mask)[0]
            
            localization_loss = (
                    torch.nn.functional.smooth_l1_loss(
                        box_transform_pred[sampled_pos_idx_mask],
                        regression_targets[sampled_pos_idx_mask],
                        beta=1 / 9,
                        reduction="sum",
                    )
                    / (sampled_idxs.numel())
            ) 

            cls_loss = torch.nn.functional.binary_cross_entropy_with_logits(cls_scores[sampled_idxs].flatten(),
                                                                            labels_for_anchors[sampled_idxs].flatten())

            rpn_output['rpn_classification_loss'] = cls_loss
            rpn_output['rpn_localization_loss'] = localization_loss
            return rpn_output


class ROIHead(nn.Module):
    r"""
    ROI head on top of ROI pooling layer for generating
    classification and box transformation predictions
    We have two fc layers followed by a classification fc layer
    and a bbox regression fc layer
    """
    
    def __init__(self, model_config, num_classes, in_channels):
        super(ROIHead, self).__init__()
        self.num_classes = num_classes
        self.roi_batch_size = model_config['roi_batch_size']
        self.roi_pos_count = int(model_config['roi_pos_fraction'] * self.roi_batch_size)
        self.iou_threshold = model_config['roi_iou_threshold']
        self.low_bg_iou = model_config['roi_low_bg_iou']
        self.nms_threshold = model_config['roi_nms_threshold']
        self.topK_detections = model_config['roi_topk_detections']
        self.low_score_threshold = model_config['roi_score_threshold']
        self.pool_size = model_config['roi_pool_size']
        self.fc_inner_dim = model_config['fc_inner_dim']
        
        self.fc6 = nn.Linear(in_channels * self.pool_size * self.pool_size, self.fc_inner_dim)
        self.fc7 = nn.Linear(self.fc_inner_dim, self.fc_inner_dim)
        self.cls_layer = nn.Linear(self.fc_inner_dim, self.num_classes)
        self.bbox_reg_layer = nn.Linear(self.fc_inner_dim, self.num_classes * 4)
        
        torch.nn.init.normal_(self.cls_layer.weight, std=0.01)
        torch.nn.init.constant_(self.cls_layer.bias, 0)

        torch.nn.init.normal_(self.bbox_reg_layer.weight, std=0.001)
        torch.nn.init.constant_(self.bbox_reg_layer.bias, 0)
    
    def assign_target_to_proposals(self, proposals, gt_boxes, gt_labels):
        r"""
        Given a set of proposals and ground truth boxes and their respective labels.
        Use IOU to assign these proposals to some gt box or background
        :param proposals: (number_of_proposals, 4)
        :param gt_boxes: (number_of_gt_boxes, 4)
        :param gt_labels: (number_of_gt_boxes)
        :return:
            labels: (number_of_proposals)
            matched_gt_boxes: (number_of_proposals, 4)
        """
        # Get IOU Matrix between gt boxes and proposals
        iou_matrix = get_iou(gt_boxes, proposals)
        # For each gt box proposal find best matching gt box
        best_match_iou, best_match_gt_idx = iou_matrix.max(dim=0)
        background_proposals = (best_match_iou < self.iou_threshold) & (best_match_iou >= self.low_bg_iou)
        ignored_proposals = best_match_iou < self.low_bg_iou
        
        # Update best match of low IOU proposals to -1
        best_match_gt_idx[background_proposals] = -1
        best_match_gt_idx[ignored_proposals] = -2
        
        # Get best marching gt boxes for ALL proposals
        # Even background proposals would have a gt box assigned to it
        # Label will be used to ignore them later
        matched_gt_boxes_for_proposals = gt_boxes[best_match_gt_idx.clamp(min=0)]
        
        # Get class label for all proposals according to matching gt boxes
        labels = gt_labels[best_match_gt_idx.clamp(min=0)]
        labels = labels.to(dtype=torch.int64)
        
        # Update background proposals to be of label 0(background)
        labels[background_proposals] = 0
        
        # Set all to be ignored anchor labels as -1(will be ignored)
        labels[ignored_proposals] = -1
        
        return labels, matched_gt_boxes_for_proposals
    
    def forward(self, feat, proposals, image_shape, target):
        r"""
        Main method for ROI head that does the following:
        1. If training assign target boxes and labels to all proposals
        2. If training sample positive and negative proposals
        3. If training get bbox transformation targets for all proposals based on assignments
        4. Get ROI Pooled features for all proposals
        5. Call fc6, fc7 and classification and bbox transformation fc layers
        6. Compute classification and localization loss

        :param feat:
        :param proposals:
        :param image_shape:
        :param target:
        :return:
        """
        if self.training and target is not None:
            # Add ground truth to proposals
            proposals = torch.cat([proposals, target['bboxes'][0]], dim=0)
            
            gt_boxes = target['bboxes'][0]
            gt_labels = target['labels'][0]
            
            labels, matched_gt_boxes_for_proposals = self.assign_target_to_proposals(proposals, gt_boxes, gt_labels)
            
            sampled_neg_idx_mask, sampled_pos_idx_mask = sample_positive_negative(labels,
                                                                                  positive_count=self.roi_pos_count,
                                                                                  total_count=self.roi_batch_size)
            
            sampled_idxs = torch.where(sampled_pos_idx_mask | sampled_neg_idx_mask)[0]
            
            # Keep only sampled proposals
            proposals = proposals[sampled_idxs]
            labels = labels[sampled_idxs]
            matched_gt_boxes_for_proposals = matched_gt_boxes_for_proposals[sampled_idxs]
            regression_targets = boxes_to_transformation_targets(matched_gt_boxes_for_proposals, proposals)
            # regression_targets -> (sampled_training_proposals, 4)
            # matched_gt_boxes_for_proposals -> (sampled_training_proposals, 4)
        
        # Get desired scale to pass to roi pooling function
        # For vgg16 case this would be 1/16 (0.0625)
        size = feat.shape[-2:]
        possible_scales = []
        for s1, s2 in zip(size, image_shape):
            approx_scale = float(s1) / float(s2)
            scale = 2 ** float(torch.tensor(approx_scale).log2().round())
            possible_scales.append(scale)
        assert possible_scales[0] == possible_scales[1]
        
        # ROI pooling and call all layers for prediction
        proposal_roi_pool_feats = torchvision.ops.roi_pool(feat, [proposals],
                                                           output_size=self.pool_size,
                                                           spatial_scale=possible_scales[0])
        proposal_roi_pool_feats = proposal_roi_pool_feats.flatten(start_dim=1)
        box_fc_6 = torch.nn.functional.relu(self.fc6(proposal_roi_pool_feats))
        box_fc_7 = torch.nn.functional.relu(self.fc7(box_fc_6))
        cls_scores = self.cls_layer(box_fc_7)
        box_transform_pred = self.bbox_reg_layer(box_fc_7)
        # cls_scores -> (proposals, num_classes)
        # box_transform_pred -> (proposals, num_classes * 4)
        ##############################################
        
        num_boxes, num_classes = cls_scores.shape
        box_transform_pred = box_transform_pred.reshape(num_boxes, num_classes, 4)
        frcnn_output = {}
        if self.training and target is not None:
            classification_loss = torch.nn.functional.cross_entropy(cls_scores, labels)
            
            # Compute localization loss only for non-background labelled proposals
            fg_proposals_idxs = torch.where(labels > 0)[0]
            # Get class labels for these positive proposals
            fg_cls_labels = labels[fg_proposals_idxs]
            
            localization_loss = torch.nn.functional.smooth_l1_loss(
                box_transform_pred[fg_proposals_idxs, fg_cls_labels],
                regression_targets[fg_proposals_idxs],
                beta=1/9,
                reduction="sum",
            )
            localization_loss = localization_loss / labels.numel()
            frcnn_output['frcnn_classification_loss'] = classification_loss
            frcnn_output['frcnn_localization_loss'] = localization_loss
        
        if self.training:
            return frcnn_output
        else:
            device = cls_scores.device
            # Apply transformation predictions to proposals
            pred_boxes = apply_regression_pred_to_anchors_or_proposals(box_transform_pred, proposals)
            pred_scores = torch.nn.functional.softmax(cls_scores, dim=-1)
            
            # Clamp box to image boundary
            pred_boxes = clamp_boxes_to_image_boundary(pred_boxes, image_shape)
            
            # create labels for each prediction
            pred_labels = torch.arange(num_classes, device=device)
            pred_labels = pred_labels.view(1, -1).expand_as(pred_scores)
            
            # remove predictions with the background label
            pred_boxes = pred_boxes[:, 1:]
            pred_scores = pred_scores[:, 1:]
            pred_labels = pred_labels[:, 1:]
            
            # pred_boxes -> (number_proposals, num_classes-1, 4)
            # pred_scores -> (number_proposals, num_classes-1)
            # pred_labels -> (number_proposals, num_classes-1)
            
            # batch everything, by making every class prediction be a separate instance
            pred_boxes = pred_boxes.reshape(-1, 4)
            pred_scores = pred_scores.reshape(-1)
            pred_labels = pred_labels.reshape(-1)
            
            pred_boxes, pred_labels, pred_scores = self.filter_predictions(pred_boxes, pred_labels, pred_scores)
            frcnn_output['boxes'] = pred_boxes
            frcnn_output['scores'] = pred_scores
            frcnn_output['labels'] = pred_labels
            return frcnn_output
    
    def filter_predictions(self, pred_boxes, pred_labels, pred_scores):
        r"""
        Method to filter predictions by applying the following in order:
        1. Filter low scoring boxes
        2. Remove small size boxes∂
        3. NMS for each class separately
        4. Keep only topK detections
        :param pred_boxes:
        :param pred_labels:
        :param pred_scores:
        :return:
        """
        # remove low scoring boxes
        keep = torch.where(pred_scores > self.low_score_threshold)[0]
        pred_boxes, pred_scores, pred_labels = pred_boxes[keep], pred_scores[keep], pred_labels[keep]
        
        # Remove small boxes
        min_size = 16
        ws, hs = pred_boxes[:, 2] - pred_boxes[:, 0], pred_boxes[:, 3] - pred_boxes[:, 1]
        keep = (ws >= min_size) & (hs >= min_size)
        keep = torch.where(keep)[0]
        pred_boxes, pred_scores, pred_labels = pred_boxes[keep], pred_scores[keep], pred_labels[keep]
        
        # Class wise nms
        keep_mask = torch.zeros_like(pred_scores, dtype=torch.bool)
        for class_id in torch.unique(pred_labels):
            curr_indices = torch.where(pred_labels == class_id)[0]
            curr_keep_indices = torch.ops.torchvision.nms(pred_boxes[curr_indices],
                                                          pred_scores[curr_indices],
                                                          self.nms_threshold)
            keep_mask[curr_indices[curr_keep_indices]] = True
        keep_indices = torch.where(keep_mask)[0]
        post_nms_keep_indices = keep_indices[pred_scores[keep_indices].sort(descending=True)[1]]
        keep = post_nms_keep_indices[:self.topK_detections]
        pred_boxes, pred_scores, pred_labels = pred_boxes[keep], pred_scores[keep], pred_labels[keep]
        return pred_boxes, pred_labels, pred_scores


class FasterRCNN(nn.Module):
    def __init__(self, model_config, num_classes):
        super(FasterRCNN, self).__init__()
        self.model_config = model_config
        vgg16 = torchvision.models.vgg16(pretrained=True)
        self.backbone = vgg16.features[:-1]
        self.rpn = RegionProposalNetwork(model_config['backbone_out_channels'],
                                         scales=model_config['scales'],
                                         aspect_ratios=model_config['aspect_ratios'],
                                         model_config=model_config)
        self.roi_head = ROIHead(model_config, num_classes, in_channels=model_config['backbone_out_channels'])
        for layer in self.backbone[:10]:
            for p in layer.parameters():
                p.requires_grad = False
        self.image_mean = [0.485, 0.456, 0.406]
        self.image_std = [0.229, 0.224, 0.225]
        self.min_size = model_config['min_im_size']
        self.max_size = model_config['max_im_size']
    
    def normalize_resize_image_and_boxes(self, image, bboxes):
        dtype, device = image.dtype, image.device
        
        # Normalize
        mean = torch.as_tensor(self.image_mean, dtype=dtype, device=device)
        std = torch.as_tensor(self.image_std, dtype=dtype, device=device)
        image = (image - mean[:, None, None]) / std[:, None, None]
        #############
        
        # Resize to 1000x600 such that lowest size dimension is scaled upto 600
        # but larger dimension is not more than 1000
        # So compute scale factor for both and scale is minimum of these two
        h, w = image.shape[-2:]
        im_shape = torch.tensor(image.shape[-2:])
        min_size = torch.min(im_shape).to(dtype=torch.float32)
        max_size = torch.max(im_shape).to(dtype=torch.float32)
        scale = torch.min(float(self.min_size) / min_size, float(self.max_size) / max_size)
        scale_factor = scale.item()
        
        # Resize image based on scale computed
        image = torch.nn.functional.interpolate(
            image,
            size=None,
            scale_factor=scale_factor,
            mode="bilinear",
            recompute_scale_factor=True,
            align_corners=False,
        )

        if bboxes is not None:
            # Resize boxes by
            ratios = [
                torch.tensor(s, dtype=torch.float32, device=bboxes.device)
                / torch.tensor(s_orig, dtype=torch.float32, device=bboxes.device)
                for s, s_orig in zip(image.shape[-2:], (h, w))
            ]
            ratio_height, ratio_width = ratios
            xmin, ymin, xmax, ymax = bboxes.unbind(2)
            xmin = xmin * ratio_width
            xmax = xmax * ratio_width
            ymin = ymin * ratio_height
            ymax = ymax * ratio_height
            bboxes = torch.stack((xmin, ymin, xmax, ymax), dim=2)
        return image, bboxes
    
    def forward(self, image, target=None):
        old_shape = image.shape[-2:]
        if self.training:
            # Normalize and resize boxes
            image, bboxes = self.normalize_resize_image_and_boxes(image, target['bboxes'])
            target['bboxes'] = bboxes
        else:
            image, _ = self.normalize_resize_image_and_boxes(image, None)
        
        # Call backbone
        feat = self.backbone(image)
        
        # Call RPN and get proposals
        rpn_output = self.rpn(image, feat, target)
        proposals = rpn_output['proposals']
        
        # Call ROI head and convert proposals to boxes
        frcnn_output = self.roi_head(feat, proposals, image.shape[-2:], target)
        if not self.training:
            # Transform boxes to original image dimensions called only during inference
            frcnn_output['boxes'] = transform_boxes_to_original_size(frcnn_output['boxes'],
                                                                     image.shape[-2:],
                                                                     old_shape)
        return rpn_output, frcnn_output


In [2]:
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from PIL import Image
import numpy as np
import os
import random
import yaml
import torch
import pandas as pd

In [1]:
import torch
from torch.utils.data import Dataset
from PIL import Image
import os
import torchvision.transforms as T

class PerImageAnnotationDataset(Dataset):
    def __init__(self, image_dir, annotation_dir, transform=None, annotation_ext=".txt"):
        """
        Args:
            image_dir (str): Path to image files.
            annotation_dir (str): Path to per-image annotation files.
            transform (callable, optional): Image transforms (should include ToTensor()).
            annotation_ext (str): Annotation file extension (e.g., ".txt").
        """
        self.image_dir = image_dir
        self.annotation_dir = annotation_dir
        self.transform = transform or T.ToTensor()
        self.annotation_ext = annotation_ext

        self.image_filenames = [f for f in os.listdir(image_dir)
                                if f.lower().endswith(('.jpg', '.jpeg', '.png'))]

    def __len__(self):
        return len(self.image_filenames)

    def __getitem__(self, idx):
        image_name = self.image_filenames[idx]
        image_path = os.path.join(self.image_dir, image_name)
        annotation_name = os.path.splitext(image_name)[0] + self.annotation_ext
        annotation_path = os.path.join(self.annotation_dir, annotation_name)

        # Load and transform image
        image = Image.open(image_path).convert("RGB")
        image = self.transform(image)  # Returns a torch.FloatTensor: [C, H, W]

        # Load bounding boxes and class labels
        boxes = []
        labels = []
        with open(annotation_path, 'r') as f:
            for line in f:
                parts = line.strip().split()
                if len(parts) != 5:
                    continue  # skip invalid lines
                x1, y1, x2, y2, class_id = map(float, parts)
                boxes.append([x1, y1, x2, y2])
                labels.append(int(class_id))

        target = {
            'bboxes': torch.tensor(boxes, dtype=torch.float32),
            'labels': torch.tensor(labels, dtype=torch.int64),
            'image_id': torch.tensor([idx])
        }

        return image, target, image_name  # Return image, target, and image name


In [ ]:
from torch.optim.lr_scheduler import MultiStepLR
from tqdm import tqdm 

def train(config_path='config.yaml', device='cuda'):
    # Read the config file #
    with open(config_path, 'r') as file:
        try:
            config = yaml.safe_load(file)
        except yaml.YAMLError as exc:
            print(exc)
    print(config)
    ########################
    
    dataset_config = config['dataset_params']
    model_config = config['model_params']
    train_config = config['train_params']
    
    seed = train_config['seed']
    torch.manual_seed(seed)
    np.random.seed(seed)
    random.seed(seed)
    if device == 'cuda':
        torch.cuda.manual_seed_all(seed)
    
    dataset= PerImageAnnotationDataset(config['dataset_params']['im_train_path'],
                                      config['dataset_params']['ann_train_path'])
    train_dataset = DataLoader(dataset,
                               batch_size=1,
                               shuffle=True,
                               num_workers=4)
    
    faster_rcnn_model = FasterRCNN(model_config,
                                   num_classes=dataset_config['num_classes'])
    faster_rcnn_model.train()
    faster_rcnn_model.to(device)

    if not os.path.exists(train_config['task_name']):
        os.mkdir(train_config['task_name'])
    optimizer = torch.optim.SGD(lr=train_config['lr'],
                                params=filter(lambda p: p.requires_grad,
                                              faster_rcnn_model.parameters()),
                                weight_decay=5E-4,
                                momentum=0.9)
    scheduler = MultiStepLR(optimizer, milestones=train_config['lr_steps'], gamma=0.1)
    
    acc_steps = train_config['acc_steps']
    num_epochs = train_config['num_epochs']
    step_count = 1

    for i in range(num_epochs):
        rpn_classification_losses = []
        rpn_localization_losses = []
        frcnn_classification_losses = []
        frcnn_localization_losses = []
        optimizer.zero_grad()
        
        for im, target, fname in tqdm(train_dataset):
            im = im.float().to(device)
            target['bboxes'] = target['bboxes'].float().to(device)
            target['labels'] = target['labels'].long().to(device)
            rpn_output, frcnn_output = faster_rcnn_model(im, target)
            
            rpn_loss = rpn_output['rpn_classification_loss'] + rpn_output['rpn_localization_loss']
            frcnn_loss = frcnn_output['frcnn_classification_loss'] + frcnn_output['frcnn_localization_loss']
            loss = rpn_loss + frcnn_loss
            
            rpn_classification_losses.append(rpn_output['rpn_classification_loss'].item())
            rpn_localization_losses.append(rpn_output['rpn_localization_loss'].item())
            frcnn_classification_losses.append(frcnn_output['frcnn_classification_loss'].item())
            frcnn_localization_losses.append(frcnn_output['frcnn_localization_loss'].item())
            loss = loss / acc_steps
            loss.backward()
            if step_count % acc_steps == 0:
                optimizer.step()
                optimizer.zero_grad()
            step_count += 1
        print('Finished epoch {}'.format(i))
        optimizer.step()
        optimizer.zero_grad()
        torch.save(faster_rcnn_model.state_dict(), os.path.join(train_config['task_name'],
                                                                train_config['ckpt_name']))
        loss_output = ''
        loss_output += 'RPN Classification Loss : {:.4f}'.format(np.mean(rpn_classification_losses))
        loss_output += ' | RPN Localization Loss : {:.4f}'.format(np.mean(rpn_localization_losses))
        loss_output += ' | FRCNN Classification Loss : {:.4f}'.format(np.mean(frcnn_classification_losses))
        loss_output += ' | FRCNN Localization Loss : {:.4f}'.format(np.mean(frcnn_localization_losses))
        print(loss_output)
        scheduler.step()
    print('Done Training...')



train()

{'dataset_params': {'im_train_path': 'data/train/train_image', 'ann_train_path': 'data/train/train_annotation', 'im_test_path': 'data/test/test_image', 'ann_test_path': 'data/test/test_annotation', 'num_classes': 8}, 'model_params': {'im_channels': 3, 'aspect_ratios': [0.5, 1, 2], 'scales': [128, 256, 512], 'min_im_size': 600, 'max_im_size': 1000, 'backbone_out_channels': 512, 'fc_inner_dim': 1024, 'rpn_bg_threshold': 0.3, 'rpn_fg_threshold': 0.7, 'rpn_nms_threshold': 0.7, 'rpn_train_prenms_topk': 12000, 'rpn_test_prenms_topk': 6000, 'rpn_train_topk': 2000, 'rpn_test_topk': 300, 'rpn_batch_size': 256, 'rpn_pos_fraction': 0.5, 'roi_iou_threshold': 0.5, 'roi_low_bg_iou': 0.0, 'roi_pool_size': 7, 'roi_nms_threshold': 0.3, 'roi_topk_detections': 100, 'roi_score_threshold': 0.05, 'roi_batch_size': 128, 'roi_pos_fraction': 0.25}, 'train_params': {'task_name': 'voc', 'seed': 1111, 'acc_steps': 1, 'num_epochs': 20, 'lr_steps': [12, 16], 'lr': 0.001, 'ckpt_name': 'faster_rcnn_voc2007.pth'}}


/home/nikos/anaconda3/lib/python3.12/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/nikos/anaconda3/lib/python3.12/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
100%|██████████| 36295/36295 [2:09:36<00:00,  4.67it/s]  


Finished epoch 0
RPN Classification Loss : 0.2127 | RPN Localization Loss : 0.1410 | FRCNN Classification Loss : 0.2793 | FRCNN Localization Loss : 0.0467


100%|██████████| 36295/36295 [2:10:00<00:00,  4.65it/s]  


Finished epoch 1
RPN Classification Loss : 0.1811 | RPN Localization Loss : 0.1285 | FRCNN Classification Loss : 0.2192 | FRCNN Localization Loss : 0.0409


100%|██████████| 36295/36295 [2:08:55<00:00,  4.69it/s]  


Finished epoch 2
RPN Classification Loss : 0.1736 | RPN Localization Loss : 0.1253 | FRCNN Classification Loss : 0.2029 | FRCNN Localization Loss : 0.0390


100%|██████████| 36295/36295 [2:06:15<00:00,  4.79it/s]  


Finished epoch 3
RPN Classification Loss : 0.1694 | RPN Localization Loss : 0.1238 | FRCNN Classification Loss : 0.1936 | FRCNN Localization Loss : 0.0377


 25%|██▍       | 8959/36295 [31:08<1:36:22,  4.73it/s]

In [ ]:
# Your code runs without errors. The previous cells have set up the environment and device.
# You can start building your model or loading data here.
